<a href="https://www.kaggle.com/aleksandrdubrovin/evaluate-model?scriptVersionId=88379943" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Description
I was working with one third part of dataset (851968 signals), I found myself running into problems with memory shortages on Kaggle’s default and decided to use powerful Google Cloud Platform virtual machine instance.
I used the virtual machine instance n1-standard-8 (8 vCPU, 30 GB memory) with a NVIDIA Tesla V100 GPU to run my training session.
This required about 7.5 hours of training time

## Import

In [ ]:
import os
import keras
import numpy as np 
import pandas as pd 
import itertools
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

## Constants

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):    
    files = {}
    for filename in filenames:        
        files[filename] = os.path.join(dirname, filename)
print(files)

# Open classes.txt
with open(files['classes.txt']) as file: 

   classes = file.read()

def str_to_list(line):
    line = line.replace('\n','')
    line = line.replace('classes = ','')
    line = eval(line)
    return line
    
classes = str_to_list(classes)
print(classes)

## Load data

In [ ]:
# Upload signals, labels, snrs
labels = np.load('/kaggle/input/deepsigio-radioml-201801a-new/labels.npy', mmap_mode = 'r')
signals = np.load('/kaggle/input/deepsigio-radioml-201801a-new/signals.npy', mmap_mode = 'r')
snrs = np.load('/kaggle/input/deepsigio-radioml-201801a-new/snrs.npy', mmap_mode = 'r')

train_acc = np.load('/kaggle/input/deepsigio-radioml-201801a-new/train_acc.npy', mmap_mode = 'r')
val_acc = np.load('/kaggle/input/deepsigio-radioml-201801a-new/val_acc.npy', mmap_mode = 'r')
train_loss = np.load('/kaggle/input/deepsigio-radioml-201801a-new/train_loss.npy', mmap_mode = 'r')
val_loss = np.load('/kaggle/input/deepsigio-radioml-201801a-new/val_loss.npy', mmap_mode = 'r')

# Upload model
model_full = load_model(files['model_full_SNR.h5'])

In [ ]:
# Split arrays for 3 parts (we take only third part of dataset of labeled signals because of the memory)
part = 3
signals = signals[::part, :, :]
labels = labels[::part, :]
snrs = snrs[::part, :]

print(signals.shape)
print(labels.shape)
print(snrs.shape)

In [ ]:
# Ndarray to array
snrs = np.ravel(snrs)
print(f"All possible SNRS: {np.unique(snrs)} db")

In [ ]:
# Masked numpy array 
c = np.ma.masked_where(snrs > 8, snrs)
msk = c.mask
# Count unique elements in array
print(np.unique(c.mask, return_counts=True))

In [ ]:
# Mask array of signals and labels (snrs > 8)
signals = signals[msk]
labels = labels[msk]

print(len(signals))

In [ ]:
# Train|test = 80|20

x_train,x_test, y_train, y_test = train_test_split(signals, labels, train_size=0.8, stratify=labels, random_state = 42)

# print(f"Количество строк в y_train по классам: {np.bincount(y_train)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_test)}")
print(x_test.shape)
print(y_test.shape)

# Train|validation|test = 64|16|20
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, stratify=y_train, random_state = 42)

# print(f"Количество строк в y_train по классам: {np.bincount(y_test)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_val)}")
print(x_train.shape)
print(y_train.shape)

## Evaluate Model on Test Data

In [ ]:
# evaluate model on test data
loss, acc = model_full.evaluate(x_test, y_test, batch_size=32)
print('EVALUATING MODEL ON TEST DATA:')
print('Test Accuracy: ', str(round(acc*100, 2)), '%')
print('\n')

The model achieved maximum accuracy of 95.72% on the clean signal dataset (SNR > 8dB).

*Note that the maximum classification accuracy of 62% if you want to test the model to its absolute limits on a mix of clean signals and signals with very high interference. Some of the signals have so much noise that they are virtually unrecognizable.*

## Confusion Matrix

In [ ]:
predictions=model_full.predict(x_test)
classes_y=np.argmax(predictions,axis=1)
conf_matrix = confusion_matrix(y_true=np.argmax(y_test,axis=1), y_pred=classes_y)

In [ ]:
nr_rows = conf_matrix.shape[0]
nr_cols = conf_matrix.shape[1]

plt.figure(figsize=(18,18), dpi= 200) 
im = plt.imshow(conf_matrix, cmap=plt.cm.Greens)
ax = plt.gca()
plt.title('Confusion Matrix', fontsize=16) 
plt.ylabel('Actual Labels', fontsize=12)
plt.xlabel('Predicted Labels', fontsize=12)
tick_marks = np.arange(len(classes))
plt.yticks(tick_marks, classes)
# plt.xticks(tick_marks, classes)

for i, j in itertools.product(range(nr_rows), range(nr_cols)):
    plt.text(j, i, conf_matrix[i, j], horizontalalignment='center',
            color='white' if conf_matrix[i, j] > conf_matrix.max()/2 else 'black')
    

# Divide existing axes and create new axes
# at bottom side of image
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax) 

## Conclusions
Reaching maximum accuracy of 62% on a mix of clean signals and signals with very high interference and 95.72% on the clean signal dataset, the model isn't perfect.

In [ ]:
history = [[train_acc, val_acc],[train_loss, val_loss]]
title = ['Model accuracy', 'Model loss']
ylabel = ['accuracy', 'loss']
fig = plt.figure(figsize=(20, 7), dpi=80)
for i, id in enumerate(title):
    plt.subplot(1, 2, i+1)
   
    plt.plot(history[i][0], label = 'train')
    plt.plot(history[i][1], label = 'val')
    plt.title(id)
    plt.xlabel('epoch')
    plt.ylabel(ylabel[i])
    plt.legend(loc='upper right')
plt.tight_layout(pad=1.7)
    

Below is an example of a signal that was mis-classified by the model (AM-SSB-WC signal was mis-classified as a 64QAM signal)

In [ ]:
idx = [17,18]
fig = plt.figure(figsize=(20, 5), dpi=80)
for i, id in enumerate(idx):
    plt.subplot(1, 2, i+1)
    plt.plot(signals[id][:, 0], color='green', label='I component')
    plt.plot(signals[id][:, 1], color='salmon', label='Q component')
    plt.title(classes[id])
    plt.xlabel('Points')
    plt.ylabel('Amplitude')
    plt.legend(loc='upper right')
plt.tight_layout(pad=1.7)